In [1]:
import numpy as np
import astropy.units as units
import astropy.constants as csts
import scipy.constants as constants

In [2]:
@units.quantity_input
def calc_D_eff(Gamma: units.Hz, frequency: units.Hz):
    c = csts.c
    e0 = csts.eps0
    hbar = csts.hbar
    return np.sqrt( (Gamma) * 3*np.pi * e0 *hbar * c**3 / (2*np.pi * frequency)**3 ).decompose()

In [3]:
# Calculate effective dipole moment for the optical transitions
Gamma = units.Hz/100e-9 #Natural linewidth in 2*pi*Hz
f = units.Hz * csts.c.value/271.7e-9
D_eff = calc_D_eff(Gamma, f)
D_eff

<Quantity 2.66755059e-30 A m s>

In [4]:
# Oskari uses Hz/(V/cm), assuming [mmomentum] = h/[length]
conv_Ams_to_HzVcm = 1/constants.hbar * units.Hz/(units.V/units.cm)
D_eff_HzVcm = D_eff.value*conv_Ams_to_HzVcm
D_eff_HzVcm

<Quantity 25295.10597251 cm Hz / V>

The Rabi frequency is defined as  
$\Omega_{i,j} = \frac{\vec{d}_{i,j}\cdot \vec{E}_0}{\hbar}$  
where $\vec{d}_{i,j}$ is the transition dipole and $\vec{E}_0$ is the vector electric field magnitude.

The laser intensity given by $I = \frac{cn\epsilon_0}{2}|E|^2$, which is converted to power by integrating over the area of the laser beam.

In [5]:
@units.quantity_input
def calculate_power_needed(Omega: units.Hz, ME, fwhm: units.m, D_TlF: units.Hz/(units.V/units.cm)):
    """
    Function to calculate the power required to get peak Rabi rate Omega
    for a transition with given matrix element with a gaussian spatial profile
    """
    c = csts.c
    epsilon_0 = csts.eps0
    
    #Calculate the electric field required
    # need the factor 2pi due to the conversion to Hz
    E =  Omega/(ME*D_TlF)

    #Convert E to peak intensity
    I = 1/2 * c * epsilon_0 * E**2

    #Convert FWHM to standard deviation
    sigma = fwhm/(2*np.sqrt(2*np.log(2)))

    #Convert power to amplitude of the Gaussian
    P = I * (2*np.pi *sigma**2)

    return P.to(units.W)

In [6]:
# ME_main is the coupling element from the main ground state to the main excited state
ME_main = -0.39444661441301954+0j
calculate_power_needed(4.5*Gamma*ME_main.real, ME_main.real, 2.06e-3*units.m, D_eff_HzVcm)

<Quantity 201.97160624 W>

In [7]:
@units.quantity_input
def calculate_power_needed(Omega: units.Hz, ME, fwhm: units.m, D_TlF: units.A*units.s*units.m):
    """
    Function to calculate the power required to get peak Rabi rate Omega
    for a transition with given matrix element with a gaussian spatial profile
    """
    c = csts.c
    epsilon_0 = csts.eps0
    
    #Calculate the electric field required
    E =  Omega*csts.hbar/(ME*D_TlF)

    #Convert E to peak intensity
    I = 1/2 * c * epsilon_0 * E**2

    #Convert FWHM to standard deviation
    sigma = fwhm/(2*np.sqrt(2*np.log(2)))

    #Convert power to amplitude of the Gaussian
    P = I * (2*np.pi *sigma**2)

    return P.to(units.W)

In [8]:
calculate_power_needed(4.5*Gamma*ME_main.real, ME_main.real, 2.06e-3*units.m, D_eff)

<Quantity 0.02019716 W>

In [9]:
(np.pi*2)**2

39.47841760435743

Konrad's estimate uses a uniform circle to estimate the power, which corresponds to a $\sqrt{2}$ factor in estimated Rabi rate.
For $\Omega = 4.5\Gamma$ he retrieves 40 mW UV power.

In [10]:
calculate_power_needed(4.5*Gamma*np.sqrt(2)*ME_main.real, ME_main.real, 2.06e-3*units.m, D_eff)

<Quantity 0.04039432 W>

In [11]:
@units.quantity_input
def calculate_rabi_konrad(power: units.W, diameter: units.m, D_TlF: units.A*units.s*units.m):
    c = csts.c
    hbar = csts.hbar
    eps0 = csts.eps0
    return (D_TlF*np.sqrt(8*power/(np.pi*c*eps0*diameter**2))/hbar).to('Hz')

In [12]:
calculate_rabi_konrad(40*units.mW, 3.5*units.mm, D_eff)

<Quantity 44769659.90522188 Hz>

In [13]:
# Konrads conversion from power to Rabi rate Omega
c = constants.c
eps0 = constants.epsilon_0
hbar = constants.hbar
diam_L = 3.5e-3
laser_power = 40e-3 # mW
konrad_rabi = (constants.elementary_charge*csts.a0*0.32).value * np.sqrt(8*laser_power/(np.pi*c*eps0*diam_L**2))/hbar
print(f'Ω = {konrad_rabi:.2f} Hz')
print(f'Ω = {konrad_rabi/Gamma.value:.2f} Γ')

Ω = 45533667.93 Hz
Ω = 4.55 Γ
